In [9]:
!pip install torchvision==0.16.0 torchaudio==2.1.0 torchdata==0.7.0 torchtext==0.16.0

import torch
import torchvision

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print("pytorch", torch.__version__)
print("torchvision", torchvision.__version__)


pytorch 2.1.0+cu121
torchvision 0.16.0+cu121


In [10]:
import numpy as np
from torchvision import tv_tensors
from torchvision.datasets.vision import VisionDataset
from torchvision.datasets.utils import download_and_extract_archive, verify_str_arg
from torchvision.ops import box_area

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import collections
import os
from xml.etree.ElementTree import Element as ET_Element

try:
    from defusedxml.ElementTree import parse as ET_parse
except ImportError:
    from xml.etree.ElementTree import parse as ET_parse
from typing import Any, Callable, Dict, List, Optional, Tuple


class FracAtlasDetection(VisionDataset):
    """FracAtlas dataset."""

    def __init__(
        self,
        root: str,
        image_set: str = "train",
        # download: bool = False,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        transforms: Optional[Callable] = None,
    ):
        super().__init__(root, transforms, transform, target_transform)

        valid_image_sets = ["test", "train", "val"]
        self.image_set = verify_str_arg(image_set, "image_set", valid_image_sets)

        self.url = "https://figshare.com/ndownloader/files/41725659"
        self.filename = "fracatlas.zip"

        # if download:
        if not os.path.isdir("data/FracAtlas"):
            os.makedirs("data", exist_ok=True)
            download_and_extract_archive(self.url, os.path.dirname(self.root), filename=self.filename, remove_finished=True)
            for subdir in ["Fractured", "Non_fractured"]:
              dirpath = os.path.join(self.root, "images")
              subdirpath = os.path.join(dirpath, subdir)
              for f in os.listdir(subdirpath):
                if not f.lower().endswith(".jpg"): continue
                os.rename(os.path.join(subdirpath, f), os.path.join(dirpath, f))
              os.rmdir(subdirpath)
        if not os.path.isdir(self.root):
            raise RuntimeError(
                "Dataset not found or corrupted. You can use download=True to download it"
            )

        splits_dir = os.path.join(self.root, "Utilities", "Fracture Split")
        splits_f = os.path.join(splits_dir, image_set.rstrip("\n") + ".csv")
        with open(os.path.join(splits_f)) as f:
            file_names = [
                os.path.splitext(x.strip())[0]
                for x in f.readlines()
                if x.strip() != "image_id"
            ]

        image_dir = os.path.join(self.root, "images")
        self.images = [os.path.join(image_dir, x + ".jpg") for x in file_names]

        target_dir = os.path.join(self.root, "Annotations", "PASCAL VOC")
        self.targets = [os.path.join(target_dir, x + ".xml") for x in file_names]

        assert len(self.images) == len(self.targets)

    @property
    def annotations(self) -> List[str]:
        return self.targets

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is a dictionary of the XML tree.
        """
        img = Image.open(self.images[index]).convert("RGB")
        img = tv_tensors.Image(img)
        voc_dict = self.parse_voc_xml(ET_parse(self.annotations[index]).getroot())
        target = self.voc_dict_to_target(index, voc_dict, img)
        # print(target)

        if self.transform is not None:
            img, target = self.transform(img, target)

        return img, target

    def __len__(self) -> int:
        return len(self.images)

    @staticmethod
    def parse_voc_xml(node: ET_Element) -> Dict[str, Any]:
        voc_dict: Dict[str, Any] = {}
        children = list(node)
        if children:
            def_dic: Dict[str, Any] = collections.defaultdict(list)
            for dc in map(FracAtlasDetection.parse_voc_xml, children):
                for ind, v in dc.items():
                    def_dic[ind].append(v)
            if node.tag == "annotation":
                def_dic["object"] = [def_dic["object"]]
            voc_dict = {
                node.tag: {
                    ind: v[0] if len(v) == 1 else v for ind, v in def_dic.items()
                }
            }
        if node.text:
            text = node.text.strip()
            if not children:
                voc_dict[node.tag] = text
        return voc_dict

    def voc_dict_to_target(self, index: int, item, img):
        width = int(item["annotation"]["size"]["width"])
        height = int(item["annotation"]["size"]["height"])

        num_fractures = len(item["annotation"]["object"])

        boxes = [
            (
                int(obj["bndbox"]["xmin"]),
                int(obj["bndbox"]["ymin"]),
                int(obj["bndbox"]["xmax"]),
                int(obj["bndbox"]["ymax"]),
            )
            for obj in item["annotation"]["object"]
        ]

        boxes = torch.as_tensor(boxes, dtype=torch.float32).reshape(-1, 4)

        # extract labels
        labels = torch.ones((num_fractures,), dtype=torch.int64)

        image_id = index
        area = box_area(boxes)

        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_fractures,), dtype=torch.int64)

        target = {}
        target["boxes"] = tv_tensors.BoundingBoxes(boxes, format="xyxy", canvas_size=img.shape[-2:])
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        return target


In [11]:
# import shutil
# shutil.rmtree("data")
# os.mkdir("data")


In [12]:
from torchvision.transforms import v2 as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToImage())
    if train:
        # transforms.append(T.RandomPhotometricDistort())
        # transforms.append(T.RandomZoomOut(fill={tv_tensors.Image: (0,0,0), "others": 0}))
        transforms.append(T.RandomHorizontalFlip(0.5))
    transforms.append(T.ClampBoundingBoxes())
    transforms.append(T.SanitizeBoundingBoxes())
    transforms.append(T.ToDtype(scale=True))
    return T.Compose(transforms)


In [13]:
# os.makedirs("lib", exist_ok=True)
if not os.path.exists("engine.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py -O engine.py"
    )
if not os.path.exists("utils.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py -O utils.py"
    )
if not os.path.exists("transforms.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py -O transforms.py"
    )
if not os.path.exists("coco_utils.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py -O coco_utils.py"
    )
if not os.path.exists("coco_eval.py"):
    os.system(
        "wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py -O coco_eval.py"
    )

# you also need to tweak them a bit to work with relative imports


In [14]:
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_fpn, FasterRCNN_MobileNet_V3_Large_FPN_Weights

model = fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT)
dataset = FracAtlasDetection("data/FracAtlas", image_set="train", transform=get_transform(train=True))
data_loader = torch.utils.data.DataLoader(
  dataset,
  batch_size=2,
  shuffle=True,
  # num_workers=4,
  collate_fn=lambda batch: tuple(zip(*batch)),
)

# For Training
images, targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images, targets)  # Returns losses and detections
print(output)

# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)  # Returns predictions
print(predictions[0])


{'loss_classifier': tensor(0.3677, grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.0082, grad_fn=<DivBackward0>), 'loss_objectness': tensor(0.1525, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.0083, grad_fn=<DivBackward0>)}
{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>), 'labels': tensor([], dtype=torch.int64), 'scores': tensor([], grad_fn=<IndexBackward0>)}


In [16]:
from engine import train_one_epoch, evaluate

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("device:", device)

# use our dataset and defined transformations
dataset = FracAtlasDetection('data/FracAtlas', image_set="train", transform=get_transform(train=True))
dataset_test = FracAtlasDetection('data/FracAtlas', image_set="test", transform=get_transform(train=False))

# split the dataset in train and test set
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-80])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-20:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
    # num_workers=4,
    collate_fn=lambda batch: tuple(zip(*batch)),
)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test,
    batch_size=1,
    shuffle=False,
    # num_workers=4,
    collate_fn=lambda batch: tuple(zip(*batch)),
)

model = fasterrcnn_mobilenet_v3_large_fpn(weights=FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
    params,
    lr=0.005,
    momentum=0.9,
    weight_decay=0.0005
)

# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer,
    step_size=3,
    gamma=0.1
)

# let's train it for 5 epochs
num_epochs = 5

for epoch in range(num_epochs):
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

print("That's it!")

print("Saving")
torch.save(model, 'fracatlas_fasterrcnn-mobilenetv3_v2.pth')
print("Saved")


device: cuda
Epoch: [0]  [  0/287]  eta: 0:01:12  lr: 0.000022  loss: 0.6470 (0.6470)  loss_classifier: 0.4857 (0.4857)  loss_box_reg: 0.0097 (0.0097)  loss_objectness: 0.1486 (0.1486)  loss_rpn_box_reg: 0.0031 (0.0031)  time: 0.2511  data: 0.1024  max mem: 1135
Epoch: [0]  [ 10/287]  eta: 0:00:34  lr: 0.000197  loss: 0.5759 (0.5441)  loss_classifier: 0.3707 (0.3226)  loss_box_reg: 0.0140 (0.0183)  loss_objectness: 0.1880 (0.1942)  loss_rpn_box_reg: 0.0073 (0.0090)  time: 0.1248  data: 0.0328  max mem: 1135
Epoch: [0]  [ 20/287]  eta: 0:00:30  lr: 0.000372  loss: 0.4102 (0.4394)  loss_classifier: 0.2179 (0.2462)  loss_box_reg: 0.0140 (0.0216)  loss_objectness: 0.1644 (0.1633)  loss_rpn_box_reg: 0.0077 (0.0084)  time: 0.1070  data: 0.0230  max mem: 1135
Epoch: [0]  [ 30/287]  eta: 0:00:34  lr: 0.000546  loss: 0.2626 (0.3768)  loss_classifier: 0.1126 (0.2086)  loss_box_reg: 0.0099 (0.0226)  loss_objectness: 0.0927 (0.1376)  loss_rpn_box_reg: 0.0069 (0.0082)  time: 0.1417  data: 0.0563  m